In [ ]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
#!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

# **Inference**

In [3]:
from transformers import pipeline

**Sentiment Analysis**

In [6]:
classifier = pipeline ("sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")
classifier("This restaurant is awesome")

[{'label': 'POSITIVE', 'score': 0.9998743534088135}]

**Natural Language Inference**

In [8]:
classifier  = pipeline("zero-shot-classification", model="roberta-large-mnli")
classifier ("Scuola Normale Superiore welcomes students from all over the world.", "Scuola Normale Superiore is a university.")

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'sequence': 'Scuola Normale Superiore welcomes students from all over the world.',
 'labels': ['Scuola Normale Superiore is a university.'],
 'scores': [0.8751774430274963]}

In [9]:
classifier("Scuola Normale Superiore welcomes students from all over the world.","Only people from Italy study at Scuola Normale Superiore.")

{'sequence': 'Scuola Normale Superiore welcomes students from all over the world.',
 'labels': ['Only people from Italy study at Scuola Normale Superiore.'],
 'scores': [0.00018539096345193684]}

**Text Generation**

In [10]:
generator = pipeline("text-generation")
generator("As far as I am concerned, I will", max_length = 50, do_sample = False)

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'As far as I am concerned, I will be the first to admit that I am not a fan of the idea of a "free market." I think that the idea of a free market is a bit of a stretch. I think that the idea'}]

**Named Entity Recognition (NER)**

In [12]:
pipe = pipeline("ner")
pipe("Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, therefore very close to the Manhattan Bridge which is visible from the window.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'entity': 'I-ORG',
  'score': 0.99957865,
  'index': 1,
  'word': 'Hu',
  'start': 0,
  'end': 2},
 {'entity': 'I-ORG',
  'score': 0.9909764,
  'index': 2,
  'word': '##gging',
  'start': 2,
  'end': 7},
 {'entity': 'I-ORG',
  'score': 0.9982224,
  'index': 3,
  'word': 'Face',
  'start': 8,
  'end': 12},
 {'entity': 'I-ORG',
  'score': 0.9994879,
  'index': 4,
  'word': 'Inc',
  'start': 13,
  'end': 16},
 {'entity': 'I-LOC',
  'score': 0.9994344,
  'index': 11,
  'word': 'New',
  'start': 40,
  'end': 43},
 {'entity': 'I-LOC',
  'score': 0.99931955,
  'index': 12,
  'word': 'York',
  'start': 44,
  'end': 48},
 {'entity': 'I-LOC',
  'score': 0.9993794,
  'index': 13,
  'word': 'City',
  'start': 49,
  'end': 53},
 {'entity': 'I-LOC',
  'score': 0.98625815,
  'index': 19,
  'word': 'D',
  'start': 79,
  'end': 80},
 {'entity': 'I-LOC',
  'score': 0.95142686,
  'index': 20,
  'word': '##UM',
  'start': 80,
  'end': 82},
 {'entity': 'I-LOC',
  'score': 0.9336589,
  'index': 21,
  'wor

**Summarization**

In [13]:
pipe = pipeline("summarization")
article = "New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York. A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband."
pipe(article, max_length = 130, min_length = 30, do_sample = False)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Your max_length is set to 130, but your input_length is only 55. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=27)


[{'summary_text': ' Liana Barrientos got married in Westchester County, New York, when she was 23 . A year later, she got married again, but to a different man and without divorcing her first husband .'}]

# **Fine-tuning**

In [2]:
!pip install datasets evaluate accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
main_path = "/content/drive/MyDrive/Hands-on/"

In [5]:
import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, TextClassificationPipeline

In [6]:
# Read in data
data = pd.read_csv(main_path + 'amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])

In [7]:
data.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [9]:
data['label'].value_counts()

0    500
1    500
Name: label, dtype: int64

In [10]:
train = data.sample(frac=0.8, random_state=42)

# Testing dataset
test = data.drop(train.index)

new_train = train.sample(frac=0.8, random_state=42)
val = train.drop(new_train.index)

In [11]:
print(f'The training dataset has {len(new_train)} records.')
print(f'The val dataset has {len(val)} records.')
print(f'The testing dataset has {len(test)} records.')

The training dataset has 640 records.
The val dataset has 160 records.
The testing dataset has 200 records.


In [12]:
# Convert pyhton dataframe to Hugging Face arrow dataset
train_data = Dataset.from_pandas(train)
val_data = Dataset.from_pandas(val)
test_data = Dataset.from_pandas(test)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [14]:
def tokenize_function(examples):
    return tokenizer(examples["review"], padding="max_length", truncation=True)

In [15]:
dataset_train = train_data.map(tokenize_function)
dataset_val = val_data.map(tokenize_function)
dataset_test = test_data.map(tokenize_function)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [16]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [18]:
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",          
    logging_dir='./sentiment_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,  
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [19]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.676300,0.628289,0.831250
2,0.593400,0.495229,0.918750
3,0.480700,0.406940,0.925000


TrainOutput(global_step=150, training_loss=0.5834581120808919, metrics={'train_runtime': 268.9413, 'train_samples_per_second': 8.924, 'train_steps_per_second': 0.558, 'total_flos': 631466532864000.0, 'train_loss': 0.5834581120808919, 'epoch': 3.0})

In [21]:
# Trainer evaluate
trainer.evaluate(dataset_test)

{'eval_loss': 0.4460802376270294,
 'eval_accuracy': 0.9,
 'eval_runtime': 7.0385,
 'eval_samples_per_second': 28.415,
 'eval_steps_per_second': 1.847,
 'epoch': 3.0}

In [ ]:
#trainer.state.log_history